<a href="https://colab.research.google.com/github/tgandor/urban_oculus/blob/master/colab/Validate_FRCNN_COCO2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# careful with dumb Python2 pycocotools:
# NameError: name 'unicode' is not defined in
# /usr/local/lib/python3.6/dist-packages/pycocotools/coco.py:308
# needs to be:
# if type(resFile) == str or (PYTHON_VERSION == 2 and type(resFile) == unicode):
!pip install -q -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-hkaoivdj
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-hkaoivdj
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=275279 sha256=b701b94239f04133b189ebf5217ea404500b6f063550575b2f12b51ec5e59d74
  Stored in directory: /tmp/pip-ephem-wheel-cache-q463qi82/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.0
    Uninstalling pycocotools-2.0.0:
      Successfully uninstalled pycocotools-2.0.0


In [1]:
%%time
# deps installation
try:
  import detectron2
except ImportError:
  !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
  !pip install -q -e detectron2_repo
  print('Stopping RUNTIME! Please run again.')
  import os
  os.kill(os.getpid(), 9)

CPU times: user 204 ms, sys: 40.5 ms, total: 245 ms
Wall time: 826 ms


Create example model

In [0]:
import detectron2.model_zoo
from detectron2.config import get_cfg

In [0]:
cfg = get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"

Create evaluator

In [0]:
# from detectron2.data import MetadataCatalog
# MetadataCatalog().get('coco_2014_val')
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

In [5]:
%%time
# No such file or directory: 'datasets/coco/annotations/instances_val2014.json'
import os
if not os.path.exists('datasets/coco/annotations/'):
  !mkdir -p datasets/coco           
  !wget -q http://images.cocodataset.org/annotations/annotations_trainval2017.zip
  !unzip -d datasets/coco annotations_trainval2017.zip

unzip:  cannot find or open annotations_trainval2014.zip, annotations_trainval2014.zip.zip or annotations_trainval2014.zip.ZIP.
CPU times: user 208 ms, sys: 38.4 ms, total: 246 ms
Wall time: 29.9 s


In [0]:
evaluator = COCOEvaluator('coco_2017_val', cfg, False)

Run inference / metrics

In [0]:
from detectron2.data import build_detection_test_loader
data_loader = build_detection_test_loader(cfg, 'coco_2017_val')

In [9]:
from detectron2.engine import DefaultPredictor
predictor = DefaultPredictor(cfg)
# predictor is not a model:
# results = inference_on_dataset(predictor, data_loader, evaluator)

model_final_f10217.pkl: 178MB [00:15, 11.2MB/s]                           


In [0]:
%%time
import os
if not os.path.exists('datasets/coco/val2017/'):
  !wget http://images.cocodataset.org/zips/val2017.zip
  !unzip val2017 -q -d datasets/coco/

In [12]:
%%time
results = inference_on_dataset(predictor.model, data_loader, evaluator)
results

Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=17.26s).
Accumulating evaluation results...
DONE (t=2.33s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.546
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.419
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.200
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.404
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.509
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.437
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.445
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

In [13]:
!ls

annotations_trainval2017.zip  detectron2_repo  val2017.zip
datasets		      sample_data


In [14]:
results.keys()

odict_keys(['bbox', 'segm'])

In [15]:
results['bbox']

{'AP': 37.45999238918093,
 'AP-airplane': 60.326406532074074,
 'AP-apple': 16.21554150553037,
 'AP-backpack': 13.487930022458276,
 'AP-banana': 18.662220202537185,
 'AP-baseball bat': 26.634072113048223,
 'AP-baseball glove': 33.26349612036358,
 'AP-bear': 65.25426281861931,
 'AP-bed': 37.97244362923196,
 'AP-bench': 21.51834359018652,
 'AP-bicycle': 27.31534736208186,
 'AP-bird': 32.35731670333031,
 'AP-boat': 23.538024294021298,
 'AP-book': 10.381036635573283,
 'AP-bottle': 35.72069118291466,
 'AP-bowl': 37.845595683121566,
 'AP-broccoli': 18.921609341856936,
 'AP-bus': 60.841291178176085,
 'AP-cake': 29.205287704121485,
 'AP-car': 41.70889582273138,
 'AP-carrot': 18.193311196311278,
 'AP-cat': 63.62185146672928,
 'AP-cell phone': 32.66107319914465,
 'AP-chair': 23.412093230177984,
 'AP-clock': 45.62108660363508,
 'AP-couch': 36.33790231670137,
 'AP-cow': 48.95791542521598,
 'AP-cup': 36.43059745809506,
 'AP-dining table': 23.95799656251554,
 'AP-dog': 55.16110484510733,
 'AP-donut':

In [0]:
result_file = 'coco_val2017_rcnn_3x.json'
import json
with open(result_file, 'w') as f:
  f.write(json.dumps(results['bbox']))

In [0]:
from google.colab import files

files.download(result_file)

In [1]:
!ls datasets/coco/annotations/

ls: cannot access 'datasets/coco/annotations/': No such file or directory
